# Vortex-shedding Flow Past a Cylinder (2D)

We consider the canonical problem of two-dimensional transient flow past a circular cylinder, a widely-used benchmark in computational fluid dynamics and reduced-order modeling the fluid flow is governed by the 2D incompressible Navier-Stokes equations

$$
\begin{aligned}
    \partial_t \mathbf{u} + \nabla \cdot (\mathbf{u} \otimes \mathbf{u})
    &= \nabla p + Re^{-1}\Delta \mathbf{u}
    \\
    \nabla \cdot \mathbf{u}
    &= 0,
\end{aligned}
$$

where $p \in \mathbb{R}$ denotes the pressure, $\mathbf{u} = (u_x, u_y)^\mathsf{T} \in \mathbb{R}^2$ denotes the $x$ and $y$ components of the velocity vector, and $Re$ denotes the dimensionless Reynolds number.

The problem setup, geometry, and parameterization are based on the [DFG 2D-3 benchmark in the FeatFlow suite](https://wwwold.mathematik.tu-dortmund.de/~featflow/en/benchmarks/cfdbenchmarking/flow/dfg_benchmark2_re100.html). This setup uses $Re = 100$, which represents a value that is above the critical Reynolds number for the onset of the two-dimensional vortex shedding the physical domain is depicted in the figure below:
<img src="./navier_stokes_cylinder_geometry.png" width="700" class="center">

We make use of the following standard scientific python libraries.

In [ ]:
import time
import warnings
import itertools
import h5py as h5
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

## Step 1: Acquire Training Data

We solve the 2D Navier Stokes equations numerically in double precision using finite elements with the [FEniCS](https://fenicsproject.org/) software package. Our implementation closely follows the Navier-Stokes cylinder example that can be found [here](https://a654cc05c43271a5d22f-f8befe5e0dcd44ae0dccf352c00b4664.ssl.cf5.rackcdn.com/tutorial/python/vol1/).

We use piecewise quadratic finite elements for the velocity and piecewise linear elements for the pressure. The resulting system of equations is large and sparse, making direct solvers impractical. We employ iterative Krylov subspace methods with preconditioning for improved performance. We utilize the biconjugate gradient stabilized (BiCGstab) method and the conjugate gradient (CG) method.

### Full-order Model

The finite element mesh contains $n_x = 9,477$ degrees of freedom.The discretized governing equations are integrated over the time interval $[0, 10]$ seconds in increments of $\Delta t = 10^{-3}$ seconds. This amounts to a total of $10,000$ time instants of the pressure and velocity fields. In the model reduction experiments that follow, we simplify the model by omitting the pressure term, which is justified by the fact that, for both transient and periodic flow regimes, the pressure can be implicitly determined from the velocity field through the incompressibility constraint. This means that the number of state variables is $n_s = 2$. Therefore, the size of one data snapshot is $n = n_s \times n_x = 18,954$.

The target time horizon is $[4, 10]$ seconds, corresponding to the periodic regime. Training data are collected over $[4, 7]$ seconds, whereas the remainder of the target time horizon is used for predictions beyond training. The keep the size of the training dataset reasonable, we downsample it by a factor of $10$, reducing its storage footprint to around $42$ MB for $n_t = 300$ downsampled snapshots. The training data were saved to disk in a single HDF5 file [**velocity_training_snapshots.h5**](./navier_stokes_benchmark/velocity_training_snapshots.h5). This file contains two datasets, $u\_x$ and $u\_y$, for the two downsampled velocity components of dimension $9,477 \times 300$. This amounts to a snapshot matrix $\mathbf{S} \in \mathbb{R}^{n \times n_t} = \mathbb{R}^{18,954 \times 300}$.

In [ ]:
# DoF setup
ns = 2
n = 18954
nx = int(n / ns)

nt = 300  # Number of training snapshots.

# state variable names
state_variables = ["u_x", "u_y"]

# path to the HDF5 file containing the training snapshots.
H5_training_snapshots = "./velocity_training_snapshots.h5"

# Number of time instances over the time domain of interest
# (including training and prediction).
nt_p = 600

In [ ]:
# Allocate memory for the full snapshot data.
# the full snapshot data has been saved to disk in HDF5 format
Q_global = np.zeros((n, nt))
with h5.File(H5_training_snapshots, "r") as hf:
    for j in range(ns):
        Q_global[j * nx : (j + 1) * nx, :] = hf[state_variables[j]][:]
hf.close()

## Step 2: Pre-Process Training Data

In the next step, we perform any necessary data manipulations, which can vary depending on the specific problem. This might include variable transformations for scenarios with non-polynomial governing equations or scaling transformations for those involving multiple states with varying scales.

If data transformations are used, we denote by $\mathbf{Q} \in \mathbb{R}^{m \times n_t}$ the *transformed* snapshot data on each compute core, with $m \geq n$ denoting the transformed snapshot dimension. Note that $m$ exceeds $n$ when the number of lifted variables exceeds the number of original state variables. If data transformations are not required, we employ $\mathbf{S}$ as is and use the notation $\mathbf{Q} = \mathbf{S}$ and $m = n$ for convenience.

### Data Scaling

For the considered 2D Navier-Stokes example, the only employed data transformation is snapshot centering with respect to the temporal mean over the training time horizon.

In [ ]:
# Compute the global temporal mean of each variable over the training horizon.
temporal_mean_global = np.mean(Q_global, axis=1)

# Center (in place) each variable with respect to its global temporal mean.
Q_global -= temporal_mean_global[:, np.newaxis]

## Step 3: Reduce Data Dimensionality

In the next step, we perform parallel dimensionality reduction, which represent the high-dimensional (transformed) snapshot data with global dimension $m$ in a lower-dimensional space of dimension $r$ such that $r \ll m$. Here, the reduced space is a linear subspace, spanned by the column vectors forming the $r$-dimensional POD basis. This step is typically the most computationally and memory-intensive part of the standard, serial OpInf formulation.

### Leveraging the method of snapshots

Starting from the method of snapshots, we can efficiently represent the high-dimensional snapshot data in the low-dimensional subspace spanned by the $r$-dimensional POD basis vectors without explicitly having to compute the POD basis and without introducing approximations by using, for example, a randomized SVD technique. We start by computing the Gram matrix $\mathbf{D} = \mathbf{Q}^\top \mathbf{Q} \in \mathbb{R}^{n_t \times n_t}$.

Consider the thin singular value decomposition (SVD) of $\mathbf{Q}$ computed at a cost in $\mathcal{O}(mn_t^2)$:
$$ \begin{equation}
    \mathbf{Q} = \mathbf{V} \mathbf{\Sigma} \mathbf{W}^\top,
\end{equation}
$$
where $\mathbf{V} \in \mathbb{R}^{m \times n_t}$ contains the left singular vectors, $\mathbf{\Sigma} \in \mathbb{R}^{n_t \times n_t}$ is a diagonal matrix comprising the singular values in non-decreasing order $\sigma_1 \geq \sigma_2 \geq \ldots \geq \sigma_{n_t}$, where $\sigma_j$ denotes the $j$th singular value, and $\mathbf{W} \in \mathbb{R}^{n_t \times n_t}$ contains the right singular vectors. The rank-$r$ POD basis $\mathbf{V}_r \in \mathbb{R}^{m \times r}$ is obtained from the first $r$ columns of $\mathbf{V}$ corresponding to the $r$ largest singular values.

In the standard OpInf formulation, the low-dimensional representation of $\mathbf{Q}$ in the linear subspace spanned by the columns of $\mathbf{V}_r$ is computed as
$$
\begin{equation}
\hat{\mathbf{Q}} = \mathbf{V}_r^\top \mathbf{Q} \in \mathbb{R}^{r \times n_t}
\end{equation}
$$
However, we have that
$$
\begin{equation} 
    \mathbf{D} = \mathbf{Q}^\top \mathbf{Q} = \mathbf{W} \mathbf{\Sigma} \mathbf{V}^\top \mathbf{V} \mathbf{\Sigma} \mathbf{W}^\top = \mathbf{W} \mathbf{\Sigma}^2 \mathbf{W}^\top \Rightarrow \mathbf{D} \mathbf{W} = \mathbf{W} \mathbf{\Sigma}^2,
\end{equation}
$$
which implies that the eigenvalues of $\mathbf{D}$ are the squared singular values of $\mathbf{Q}$, and the eigenvectors of $\mathbf{D}$ are equivalent (up to a sign change) to the right singular vectors of $\mathbf{Q}$. We then compute the eigenpairs $\{(\lambda_k, \mathbf{u}_k)\}_{k=1}^{n_t}$ of $\mathbf{D}$, where $\lambda_k$ are the real and non-negative eigenvalues and $\mathbf{u}_k \in \mathbb{R}^{n_t}$ denote the corresponding eigenvectors. We also ensure that the eigenpairs are arranged such that $\lambda_1 \geq \lambda_2 \geq \ldots \geq \lambda_{n_t}$
The rank-$r$ POD basis can be also computed as
$$
\begin{equation}
    \mathbf{V}_r = \mathbf{Q} \mathbf{W}_r \mathbf{\Sigma}_r^{-1} = \mathbf{Q} \mathbf{U}_r \mathbf{\Lambda}_r^{-\frac{1}{2}}, 
\end{equation}
$$
where $\mathbf{U}_r = \begin{bmatrix} \mathbf{u}_1 \vert \mathbf{u}_2 \vert \dots \vert \mathbf{u}_r \end{bmatrix}$ and $\mathbf{\Lambda}_r = \mathrm{diag}(\lambda_1, \lambda_2, \ldots, \lambda_r)$.
The above expression implies that
$$
\begin{equation}
    \hat{\mathbf{Q}} = \mathbf{V}_r^\top \mathbf{Q} = \left(\mathbf{Q} \mathbf{U}_r \mathbf{\Lambda}_r^{-\frac{1}{2}}\right)^\top \mathbf{Q} = \mathbf{T}_r^\top \mathbf{Q}^\top \mathbf{Q} = \mathbf{T}_r^\top \mathbf{D},
\end{equation}
$$
where we used the notation $\mathbf{T}_r =  \mathbf{U}_r \mathbf{\Lambda}_r^{-\frac{1}{2}} \in \mathbb{R}^{n_t \times r}$.

### Efficient data dimensionality reduction without explicitly requiring the POD basis

Therefore, the representation of the high-dimensional (transformed) snapshots in the low-dimensional linear subspace spanned by the rank-$r$ POD basis vectors can be efficiently computed in terms of two small matrices, $\mathbf{T}_r$ and $\mathbf{D}$, without explicitly requiring the POD basis
In our implementation, we choose the reduced dimension, $r$, such that the total retained energy corresponding to the first $r$ POD modes is $99.9 \%$, that is,
$$
\begin{equation}
    \frac{\sum_{k=1}^{r} \sigma_k^2}{\sum_{k=1}^{n_t} \sigma_k^2} \geq 0.999.
\end{equation}
$$

In [ ]:
# Compute the global Gram matrix and get its eigendecomposition.
D_global = Q_global.T @ Q_global
eigs, eigv = np.linalg.eigh(D_global)

# Order the eigenpairs by increasing eigenvalue magnitude.
sorted_indices = np.argsort(eigs)[::-1]
eigs = eigs[sorted_indices]
eigv = eigv[:, sorted_indices]

# Amount of energy to retain in the basis.
target_ret_energy = 0.999

# Determine the minimum integer r that exceeds the retained energy threshold.
ret_energy = np.cumsum(eigs) / np.sum(eigs)
r = np.argmax(ret_energy > target_ret_energy) + 1

# Compute the auxiliary Tr matrix.
Tr_global = eigv[:, :r] @ np.diag(eigs[:r] ** (-0.5))

# Efficiently compute the low-dimensional representation of the
# high-dimensional transformed snapshot data.
Qhat_global = Tr_global.T @ D_global

## Step 4: Learn Reduced Operators from Data

The Navier-Stokes equations have only linear and quadratic terms in the governing equations. We additionally have a constant term introduced into the ROM, due to centering. Since the training snapshots were downsampled in time, we employ the time-discrete formulation of OpInf.

The goal is to determine the reduced operators $\hat{\mathbf{c}} \in \mathbb{R}^{r}, \hat{\mathbf{A}} \in \mathbb{R}^{r\times r}$, and $\hat{\mathbf{H}} \in \mathbb{R}^{r\times r^2}$ defining the discrete quadratic ROM
$$
\begin{equation} 
    \hat{\mathbf{q}}[k + 1] = \hat{\mathbf{A}}\hat{\mathbf{q}}[k] + \hat{\mathbf{H}}\left(\hat{\mathbf{q}}[k] \otimes \hat{\mathbf{q}}[k] \right) + \hat{\mathbf{c}}
\end{equation}
$$
that best match the projected snapshot data in a minimum residual sense by solving the linear least-squares minimization
$$
\begin{equation}
    \mathop{\mathrm{argmin}}_{\hat{\mathbf{O}}} \left\lVert \hat{\mathbf{D}}\hat{\mathbf{O}}^{\top} - \hat{\mathbf{Q}}_2^\top \right\rVert_F^2 + \beta_{1} \left(\left\lVert\hat{\mathbf{A}}\right\rVert_F^2 + \left\lVert\hat{\mathbf{c}}\right\rVert_F^2\right) + \beta_{2} \left\lVert\hat{\mathbf{H}}\right\rVert_F^2,
\end{equation}
$$
where  $\hat{\mathbf{O}} =
\begin{bmatrix}
\hat{\mathbf{A}} \, \vert \, \hat{\mathbf{H}} \, \vert \, \hat{\mathbf{c}}
\end{bmatrix}
\in \mathbb{R}^{r \times (r + r^2 + 1)}$ denotes the unknown operators and  $\hat{\mathbf{D}} =
\begin{bmatrix}
\hat{\mathbf{Q}}_1^\top \, \vert \, \hat{\mathbf{Q}}_1^\top \otimes \hat{\mathbf{Q}}_1^\top \, \vert \, \hat{\mathbf{1}}_{n_t - 1}
\end{bmatrix}
\in \mathbb{R}^{(n_t - 1) \times (r + r^2 +1)}$ the OpInf data, $F$ denotes the Frobenius norm, and
\begin{equation} 
    \hat{\mathbf{Q}}_{1} =
     \begin{bmatrix}
\vert & \vert & & \vert\\
     \hat{\mathbf{q}}_1 &
     \hat{\mathbf{q}}_2 &
     \ldots &
     \hat{\mathbf{q}}_{n_t - 1}\\
     \vert & \vert & & \vert
     \end{bmatrix} \in \mathbb{R}^{r \times n_t-1} \quad \text{and} \quad 
     \hat{\mathbf{Q}}_{2} =
     \begin{bmatrix}
\vert & \vert & & \vert\\
     \hat{\mathbf{q}}_2 &
     \hat{\mathbf{q}}_3 &
     \ldots &
     \hat{\mathbf{q}}_{n_t}\\
     \vert & \vert & & \vert
     \end{bmatrix} \in \mathbb{R}^{r \times n_t-1}
\end{equation}

To address overfitting and other sources of error, we introduce regularization hyperparameters $\beta_{1}, \beta_{2} \in \mathbb{R}$.

### Defining some useful auxiliary functions

We start with providing the code for four auxiliary functions that we will need later on.

In [ ]:
# # Alternative to compute_Qhat_sq() used in euler.ipynb:
# def compressed_kronecker(state):
#     """Unique entries of the Kronecker product of a vector with itself."""
#     return np.concatenate(
#         [state[i] * state[: i + 1] for i in range(state.shape[0])],
#         axis=0,
#     )


def compute_Qhat_sq(Qhat):
    r"""Compute the non-redundant terms in Qhat \otimes Qhat."""

    if len(np.shape(Qhat)) == 1:
        r = np.size(Qhat)
        prods = []
        for i in range(r):
            temp = Qhat[i] * Qhat[i:]
            prods.append(temp)

        Qhat_sq = np.concatenate(tuple(prods))

    elif len(np.shape(Qhat)) == 2:
        K, r = np.shape(Qhat)
        prods = []
        for i in range(r):
            temp = (
                np.transpose(np.broadcast_to(Qhat[:, i], (r - i, K)))
                * Qhat[:, i:]
            )
            prods.append(temp)

        Qhat_sq = np.concatenate(tuple(prods), axis=1)

    else:
        print("invalid input!")

    return Qhat_sq


def compute_train_err(Qhat_train, Qtilde_train):
    """Computes the OpInf training error.

    Parameters
    ----------
    Qhat_train : (r, k) ndarray
        Reference data, the high-dimensional snapshots after
        data transformation and compression.
    Qtilde_train : (r, k) ndarray
        Solution to the OpInf ROM that should match the reference data.

    Returns
    -------
    train_error : float
        Error between the reference data and the ROM solution.
    """
    # return np.max(
    #     la.norm(Qtilde_train - Qhat_train, axis=1)
    #     / la.norm(Qhat_train, axis=1)
    # )
    return np.max(
        np.sqrt(
            np.sum((Qtilde_train - Qhat_train) ** 2, axis=1)
            / np.sum(Qhat_train**2, axis=1)
        )
    )


def solve_opinf_difference_model(initial_condition, n_steps, reduced_model):
    """Solve the discrete OpInf ROM over a prescribed trial time horizon.

    Parameters
    ----------
    initial_condition : (r,) ndarray
        Initial condition in the reduced space.
    n_steps : int
        Number of time steps to compute the ROM solution.
    reduced_model : callable
        Function for the right-hand side of the ROM.

    Returns
    -------
    contains_nans : bool
        ``True`` if the solution contains ``NaN``, ``False`` otherwise.
    Qtilde : (n_steps, r) ndarray
        Solution to the ROM.
    """

    Qtilde = np.zeros((np.size(initial_condition), n_steps))
    contains_nans = False

    Qtilde[:, 0] = initial_condition
    for i in range(n_steps - 1):
        Qtilde[:, i + 1] = reduced_model(Qtilde[:, i])

    if np.any(np.isnan(Qtilde)):
        contains_nans = True

    return contains_nans, Qtilde.T

### Finding the optimal hyperparameter values via a grid search

We employ a grid search to find the optimal hyperparameter values. This involves exploring a range of candidate values for both $\beta_{1}$ and $\beta_{2} \in \mathbb{R}$ in a nested loop. We prescribe discrete sets of candidate values $\mathcal{B}_1$ and $\mathcal{B}_2$ for the regularization parameters.

We consider eight candidate values for both $\mathcal{B}_1$ and $\mathcal{B}_2$, noting that in problems with more complex dynamics, sets with larger cardinalities (and bounds) might be necessary to ensure that the optimal pair leads to accurately inferred reduced operators. We also prescribe a tolerance for the maximum growth of the inferred reduced coefficients over the trial time horizon, which will be used to determine the optimal regularization pair. The optimal hyperparameters are chosen to minimize the training error, subject to the constraint that the inferred reduced coefficient have bounded growth over a trial time horizon $[t_{\mathrm{init}}, t_{\mathrm{trial}}]$ with $t_{\mathrm{trial}} \geq t_{\mathrm{final}}$; in our implementation, the trial time horizon is the same as the target horizon, that is, $[4, 10]$ seconds. Moreover, the solution to the OpInf least-squares minimization is determined by solving the normal equations

In [ ]:
# Define ranges for the regularization parameter pairs.
B1 = np.logspace(-10.0, 0.0, num=8)
B2 = np.logspace(-4.0, 4.0, num=8)

# Get the Cartesian product of all regularization pairs (beta1, beta2).
reg_pairs_global = list(itertools.product(B1, B2))
n_reg_global = len(reg_pairs_global)

# Set the threshold for the maximum growth of the inferred reduced
# coefficients, used for selecting the optimal regularization parameter pair.
max_growth = 1.2

# Extract left and right shifted reduced data matrices for discrete OpInf.
Qhat_1 = Qhat_global.T[:-1, :]
Qhat_2 = Qhat_global.T[1:, :]

# Column dimension of the data matrix Dhat used in discrete OpInf.
s = int(r * (r + 1) / 2)
d = r + s + 1

# Compute the non-redundant quadratic terms of Qhat_1 squared.
Qhat_1_sq = compute_Qhat_sq(Qhat_1)

# Define the constant part (due to mean shifting) for discrete OpInf.
K = Qhat_1.shape[0]
Ehat = np.ones((K, 1))

# Assemble the data matrix Dhat for the discrete OpInf learning problem.
Dhat = np.concatenate((Qhat_1, Qhat_1_sq, Ehat), axis=1)
# Compute Dhat^T Dhat for the normal equations to solve the least squares.
Dhat_2 = Dhat.T @ Dhat

# Compute the temporal mean and maximum deviation of the reduced training data.
mean_Qhat_train = np.mean(Qhat_global.T, axis=0)
max_diff_Qhat_train = np.max(np.abs(Qhat_global.T - mean_Qhat_train), axis=0)

# Loop over all regularization pairs.
best_train_err = 1e20
best_beta1, best_beta2 = None, None
for beta1, beta2 in reg_pairs_global:

    start_time_OpInf_learning = time.time()

    # Construct a regularizer that penalizes the linear and constant reduced
    # operators using beta1 and the quadratic operator using beta2.
    regg = np.zeros(d)
    regg[:r] = beta1
    regg[r : r + s] = beta2
    regg[r + s :] = beta1
    regularizer = np.diag(regg)
    Dhat_2_reg = Dhat_2 + regularizer

    # Solve the OpInf problem by solving the regularized normal equations.
    Ohat = np.linalg.solve(Dhat_2_reg, np.dot(Dhat.T, Qhat_2)).T

    # Extract the linear, quadratic, and constant reduced model operators.
    Ahat = Ohat[:, :r]
    Fhat = Ohat[:, r : r + s]
    chat = Ohat[:, r + s]

    end_time_OpInf_learning = time.time()

    # Define the OpInf reduced model.
    OpInf_red_model = lambda x: Ahat @ x + Fhat @ compute_Qhat_sq(x) + chat

    # Extract the reduced initial condition from Qhat_1.
    qhat0 = Qhat_1[0, :]

    # Compute the reduced solution over the trial time horizon, which here is the same as the target time horizon
    start_time_OpInf_eval = time.time()
    contains_nans, Qtilde_OpInf = solve_opinf_difference_model(
        initial_condition=qhat0,
        n_steps=nt_p,
        reduced_model=OpInf_red_model,
    )
    end_time_OpInf_eval = time.time()

    time_OpInf_eval = end_time_OpInf_eval - start_time_OpInf_eval

    # If the model produced an unstable solution, move on to the next
    # regularization candidates.
    if contains_nans:
        continue  # go to next iteration of the for loop.

    # If the ratio of the maximum coefficient grown exceeds the allowed
    # threshold, move on to the next regularization candidates.
    max_diff_Qhat_trial = np.max(
        np.abs(Qtilde_OpInf - mean_Qhat_train), axis=0
    )
    max_growth_trial = np.max(max_diff_Qhat_trial) / np.max(
        max_diff_Qhat_train
    )
    if max_growth_trial > max_growth:
        continue

    # At this point we know the model produced a stable solution without too
    # much growth. Compute the training error and, if it better than the
    # current best error, save the regularization, reduced solution, and
    # the learning times.
    train_err = compute_train_err(Qhat_global.T[:nt, :], Qtilde_OpInf[:nt, :])
    if train_err < best_train_err:
        best_beta1 = beta1
        best_beta2 = beta2
        best_train_err = train_err
        Qtilde_OpInf_opt = Qtilde_OpInf
        OpInf_ROM_wtime_opt = time_OpInf_eval

### Step 5: Postprocessing of the reduced solution

In the final step, we postprocess the obtained reduced solution. Specifically, we use the approximate velocity solutions to compute the vorticity 
$$
\begin{equation} 
    \omega = \frac{\partial u_y}{\partial x} - \frac{\partial u_x}{\partial y}
\end{equation}
$$
at three probe locations positioned near the mid-channel, with increasing distance from the circular cylinder, namely $(0.40, 0.20), (0.60, 0.20)$, and $(1.00, 0.20)$. The corresponding indices within each snapshot for these locations are $\{997; 1,469; 1,376\}$. 

Since estimating the vorticity requires FEniCS functionality, we saved the reference and reduced model solutions here [in this folder](./navier_stokes_benchmark/).

### We start by plotting the singular value decay and corresponding retained energy.

In [ ]:
warnings.filterwarnings("ignore")

no_kept_svals_global = 200
no_kept_svals_energy = 20
no_svals_global = range(1, no_kept_svals_global + 1)
no_svals_energy = range(1, no_kept_svals_energy + 1)

retained_energy = np.cumsum(eigs) / np.sum(eigs)
target_ret_energy = 0.999

r = np.argmax(retained_energy > target_ret_energy) + 1
ret_energy = retained_energy[r]

plt.rcParams["lines.linewidth"] = 0
plt.rc("figure", dpi=400)
plt.rc("font", family="serif")
plt.rc("legend", edgecolor="none")
plt.rcParams["figure.figsize"] = (6, 2)
plt.rcParams.update({"font.size": 5})

charcoal = [0.1, 0.1, 0.1]
color1 = "#D55E00"
color2 = "#0072B2"

fig = plt.figure()
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

plt.rc("figure", facecolor="w")
plt.rc("axes", facecolor="w", edgecolor="k", labelcolor="k")
plt.rc("savefig", facecolor="w")
plt.rc("text", color="k")
plt.rc("xtick", color="k")
plt.rc("ytick", color="k")

ax1.spines["right"].set_visible(False)
ax1.spines["top"].set_visible(False)
ax1.yaxis.set_ticks_position("left")
ax1.xaxis.set_ticks_position("bottom")

ax2.spines["right"].set_visible(False)
ax2.spines["top"].set_visible(False)
ax2.yaxis.set_ticks_position("left")
ax2.xaxis.set_ticks_position("bottom")

## plot
ax1.semilogy(
    no_svals_global,
    np.sqrt(eigs)[:no_kept_svals_global] / np.sqrt(eigs[0]),
    linestyle="-",
    lw=0.75,
    color=color1,
)
ax1.set_xlabel("index")
ax1.set_ylabel("singular values transformed data")

ax2.plot(
    no_svals_energy,
    retained_energy[:no_kept_svals_energy],
    linestyle="-",
    lw=0.75,
    color=color1,
)
ax2.set_xlabel("reduced dimension")
ax2.set_ylabel("% energy retained")
ax2.plot(
    [r, r], [0, retained_energy[r]], linestyle="--", lw=0.5, color=charcoal
)
ax2.plot(
    [0, r],
    [retained_energy[r], retained_energy[r]],
    linestyle="--",
    lw=0.5,
    color=charcoal,
)

xlim = ax1.get_xlim()
ax1.set_ylim([1e-10, 1.02e0])

x_pos_all = np.array([0, 99, 199])
labels = np.array([1, 100, 200])
ax1.set_xticks(x_pos_all)
ax1.set_xticklabels(labels)

ax2.set_xlim([0, 20])
ax2.set_ylim([0.4, 1.001])

ax2.set_xticks([1, r, 20])
ax2.set_yticks([0.5, 0.75, ret_energy, 1])
ax2.set_yticklabels([r"$50\%$", r"$75\%$", r"$99.93\%$", ""])

plt.show()

### Ploting the ROM approximate vorticity solutions at the three probe locations, $(0.40, 0.20), (0.60, 0.20)$, and $(1.00, 0.20)$

The hashed areas on the left mark the training time horizon.

In [ ]:
t_start = 4
t_end = 10
t_train = 7
dt = 1e-2

t = np.arange(t_start, t_end, dt)

ref_data = np.load("./ref_vorticity_full_time_domain.npy")

ref_data_loc1 = ref_data[400:, 0]
ref_data_loc2 = ref_data[400:, 1]
ref_data_loc3 = ref_data[400:, 2]

OpInf_data = np.load("./ROM_vorticity_probes.npy")

OpInf_data_loc1 = OpInf_data[:, 0]
OpInf_data_loc2 = OpInf_data[:, 1]
OpInf_data_loc3 = OpInf_data[:, 2]


plt.rc("figure", dpi=400)
plt.rc("font", family="serif")
plt.rc("legend", edgecolor="none")
plt.rcParams["figure.figsize"] = (8, 2)
plt.rcParams.update({"font.size": 6})

charcoal = [0.1, 0.1, 0.1]
color1 = "#D55E00"
color2 = "#0072B2"

fig = plt.figure()
ax11 = fig.add_subplot(131)
ax12 = fig.add_subplot(132, sharex=ax11)
ax13 = fig.add_subplot(133, sharex=ax11)

plt.rc("figure", facecolor="w")
plt.rc("axes", facecolor="w", edgecolor="k", labelcolor="k")
plt.rc("savefig", facecolor="w")
plt.rc("text", color="k")
plt.rc("xtick", color="k")
plt.rc("ytick", color="k")


ax11.spines["right"].set_visible(False)
ax11.spines["top"].set_visible(False)
ax11.yaxis.set_ticks_position("left")
ax11.xaxis.set_ticks_position("bottom")

ax12.spines["right"].set_visible(False)
ax12.spines["top"].set_visible(False)
ax12.yaxis.set_ticks_position("left")
ax12.xaxis.set_ticks_position("bottom")


ax13.spines["right"].set_visible(False)
ax13.spines["top"].set_visible(False)
ax13.yaxis.set_ticks_position("left")
ax13.xaxis.set_ticks_position("bottom")

ax11.plot(t, ref_data_loc1, linestyle="-", lw=1.00, color=charcoal)
ax11.plot(t, OpInf_data_loc1, linestyle="--", lw=1.00, color=color1)

ax12.plot(t, ref_data_loc2, linestyle="-", lw=1.00, color=charcoal)
ax12.plot(t, OpInf_data_loc2, linestyle="--", lw=1.00, color=color1)

ax13.plot(t, ref_data_loc3, linestyle="-", lw=1.00, color=charcoal)
ax13.plot(t, OpInf_data_loc3, linestyle="--", lw=1.00, color=color1)

ax11.axvline(x=t_train, lw=1.00, linestyle="--", color="gray")
ax12.axvline(x=t_train, lw=1.00, linestyle="--", color="gray")
ax13.axvline(x=t_train, lw=1.00, linestyle="--", color="gray")

ax11.set_title("probe 1 (0.4, 0.2)")
ax12.set_title("probe 2 (0.6, 0.2)")
ax13.set_title("probe 3 (1.0, 0.2)")

ax11.set_ylabel(r"$\omega$")

fig.supxlabel("target time horizon (seconds)", y=-0.1)

xlim = ax11.get_xlim()
ax11.set_xlim([xlim[0], 10])

ylim1 = ax11.get_ylim()
ylim2 = ax12.get_ylim()
ylim3 = ax13.get_ylim()

h1 = np.abs(ylim1[0]) + np.abs(ylim1[1])
h2 = np.abs(ylim2[0]) + np.abs(ylim2[1])
h3 = np.abs(ylim3[0]) + np.abs(ylim3[1])

rect = plt.Rectangle(
    (0, ylim1[0]),
    width=t_train,
    height=h1,
    hatch="/",
    color="grey",
    alpha=0.2,
    label="training region",
)
ax11.add_patch(rect)
rect = plt.Rectangle(
    (0, ylim2[0]),
    width=t_train,
    height=h2,
    hatch="/",
    color="grey",
    alpha=0.2,
    label="training region",
)
ax12.add_patch(rect)
rect = plt.Rectangle(
    (0, ylim3[0]),
    width=t_train,
    height=h3,
    hatch="/",
    color="grey",
    alpha=0.2,
    label="training region",
)
ax13.add_patch(rect)

x_pos_all = np.array([4, 5, 6, 7, 8, 9, 10])
labels = np.array([4, 5, 6, 7, 8, 9, 10])
ax11.set_xticks(x_pos_all)
ax11.set_xticklabels(labels)

plt.show()